In [172]:
import numpy as np
import ast
import pytz
import math
import itertools
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_selection import r_regression 
np.random.seed(1337)
df = pd.read_csv('top10K-spotify.csv',index_col=0)
df["type"].value_counts()
df=df.drop(columns=["index","city_1","district_1","district_2","city_3","district_3" ])
df = df[df['city_2'].notna()]
df = df[df['country'].notna()]
df.rename(columns = {'artist':'name'}, inplace = True)

df2=pd.read_csv('data_w_spotify.csv',index_col=None)
df2=df2.drop(columns=["isdone","id"])
df2['genres'] = df2['genres'].fillna('')

df3=df.merge(df2,on='name')

df3['genres']= df3['genres'].str.strip('[]').str.replace("'","").str.split('\s*,\s*')

def create_soup(x):
    return ' '.join(x['genres']) + ' '+ x['city_2'] + ' ' + x['country']+' ' + x['gender']+ ' ' + x['type']

df3['soup'] = df3.apply(create_soup, axis=1)
print(df3.shape,df3.columns)

dict_names = dict(zip(df3.name, df3.spotifyid))

df = df.reset_index()
indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()



def get_recommendations(name,sim,reverse):
    # print(sim[0:20])
    # print(name,sim)
    indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()
    idx = indices[name]

    # Get the pairwsie similarity scores of all artsits with that movie
    sim_scores = list(enumerate(sim[idx]))
    if not isinstance(sim_scores[0][1],np.ndarray):
        # Sort the artists based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=reverse)

        # Get the scores of the 10 most similar artsits
        sim_scores = sim_scores[1:21]

        # Get the movie indices
        indices = [i[0] for i in sim_scores]
        scores = [round(i[1]*100, 2) for i in sim_scores]

        # Return the top 20 most similar artsits
        top_10=df3['name'].iloc[indices]
        spot_top10=[]
        spot_top10_names=[]
        for item in top_10:
            spot_top10_names.append(item)
            spot_top10.append(dict_names[str(item)])

        artist=dict_names[str(df3['name'].iloc[idx])]
        
        wantedDict = {
            'mainId': artist,
            'top10': spot_top10,
            'top10Names': spot_top10_names,
            'scores': scores
        }
        return wantedDict
    return {
        'top10': ['wrong format'],
        'top10Names': ['wrong format'],
    }




(1416, 11) Index(['name', 'gender', 'age', 'type', 'country', 'city_2', 'spotifyid',
       'genres', 'popularity', 'followers', 'soup'],
      dtype='object')


In [76]:
def CVectorizer():
    count = CountVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def CosSim(x):
    return (cosine_similarity(x,x),True)

In [77]:
def TfidVectorizer():
    count = TfidfVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def EuDSim(x):
    return (euclidean_distances(x,), False)

In [190]:
spot1 = pd.read_csv('spotify_results1.csv')
spot2 = pd.read_csv('spotify_results2.csv')
spot3 = pd.read_csv('spotify_results3.csv')
spotDict = {}

i = 0
for index in spot1:
    if i != 0:
        spotDict[str(index)] = list(spot1[index].to_numpy())
    i+=1

i = 0
for index in spot2:
    if i != 0:
        spotDict[str(index)] = list(spot2[index].to_numpy())
    i+=1

i = 0
for index in spot3:
    if i != 0:
        spotDict[str(index)] = list(spot3[index].to_numpy())
    i+=1

matrix=CVectorizer()
sim,reverse = EuDSim(matrix)

noCutEuDSim = []

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
    percentage = (float(count)/20.0)
    noCutEuDSim.append(percentage)

print(noCutEuDSim)

0
0
0
0
0
0
0
3
0
0
2
1
1
4
0
4
0
1
1
4
0
3
3
0
0
4
5
1
2
3
3
0
0
0
2
1
1
3
1
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
5
0
1
0
1
0
0
0
3
0
1
0
5
5
1
0
1
0
0
0
0
0
0
0
0
2
0
2
0
0
0
0
0
1
0
0
0
0
0
2
2
1
1
1
0
1
2
0
5
2
2
3
5
0
0
3
1
0
0
0
0
0
3
2
1
3
1
0
1
0
0
5
0
0
4
2
0
5
0
0
4
1
1
0
0
4
4
0
0
0
1
1
1
0
3
3
1
5
5
1
2
1
4
3
1
3
3
4
0
0
3
2
0
1
1
4
3
0
0
0
0
1
3
1
1
0
0
0
4
0
0
1
3
1
1
0
2
0
0
0
1
0
3
0
0
2
0
2
0
0
0
0
0
0
1
7
0
1
2
2
0
1
0
1
3
0
6
0
0
0
2
0
0
1
0
0
0
1
0
3
5
3
3
0
0
0
1
1
0
0
0
7
1
0
1
1
3
4
0
0
1
0
1
0
2
0
0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.1, 0.05, 0.05, 0.2, 0.0, 0.2, 0.0, 0.05, 0.05, 0.2, 0.0, 0.15, 0.15, 0.0, 0.0, 0.2, 0.25, 0.05, 0.1, 0.15, 0.15, 0.0, 0.0, 0.0, 0.1, 0.05, 0.05, 0.15, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.15, 0.0, 0.05, 0.0, 0.25, 0.25, 0.05, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [189]:
matrix=CVectorizer()
sim,reverse = EuDSim(matrix)

cutEuDSimCv= []

allowedNames = list(indices.index)

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    names = wanted_dict['top10Names']
    if names[0] != 'wrong format':
        for name in names:
            if name not in allowedNames:
                i = names.index(name)
                ids.pop(i)
        count = 0
        print(len(ids))
        for id in ids:
            if id in spotDict[ind]:
                count += 1
        percentage = (count/len(ids))
        cutEuDSimCv.append(percentage)

print(cutEuDSimCv)

20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15, 0.0, 0.0, 0.1, 0.05, 0.05, 0.2, 0.0, 0.2, 0.0, 0.05, 0.05, 0.2, 0.0, 0.15, 0.15, 0.0, 0.0, 0.2, 0.25, 0.05, 0.1, 0.15, 0.15, 0.0, 0.0, 0.0, 0.1, 0.05, 0.05, 0.15, 0.05, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 

In [162]:
matrix=TfidVectorizer()
sim,reverse = EuDSim(matrix)

noCutEuDTfSim = []

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
    percentage = (count/20)
    noCutEuDTfSim.append(percentage)

print(noCutEuDTfSim)

[0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.25, 0.0, 0.05, 0.15, 0.1, 0.05, 0.2, 0.0, 0.2, 0.0, 0.05, 0.05, 0.2, 0.0, 0.15, 0.05, 0.0, 0.0, 0.2, 0.25, 0.0, 0.2, 0.15, 0.2, 0.05, 0.0, 0.0, 0.1, 0.05, 0.05, 0.25, 0.1, 0.1, 0.0, 0.0, 0.0, 0.15, 0.2, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.15, 0.05, 0.1, 0.05, 0.05, 0.0, 0.05, 0.0, 0.2, 0.0, 0.05, 0.0, 0.2, 0.15, 0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.15, 0.0, 0.1, 0.05, 0.1, 0.15, 0.0, 0.0, 0.0, 0.05, 0.2, 0.05, 0.0, 0.0, 0.1, 0.0, 0.1, 0.2, 0.0, 0.0, 0.05, 0.0, 0.05, 0.15, 0.0, 0.2, 0.05, 0.1, 0.15, 0.15, 0.1, 0.0, 0.15, 0.05, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.1, 0.05, 0.1, 0.05, 0.0, 0.05, 0.0, 0.0, 0.05, 0.05, 0.05, 0.2, 0.1, 0.0, 0.25, 0.05, 0.0, 0.2, 0.05, 0.1, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.05, 0.05, 0.0, 0.15, 0.15, 0.05, 0.2, 0.45, 0.0, 0.0, 0.1, 0.25, 0.2, 0.0, 0.15, 0.2, 0.3, 0.0, 0.1, 0.1, 0.2, 0.0, 0.05, 0.05, 0.2, 0.15, 0.05, 0.1, 0.0, 0.0, 0.05, 0.15, 0.1, 0.1, 0.0, 0.05, 0.0, 0.1, 0.0, 0

In [188]:
matrix=TfidVectorizer()
sim,reverse = EuDSim(matrix)

cutEuDSimTf= []

allowedNames = list(indices.index)

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    names = wanted_dict['top10Names']
    if names[0] != 'wrong format':
        for name in names:
            if name not in allowedNames:
                i = names.index(name)
                ids.pop(i)
        count = 0
        print(len(ids))
        for id in ids:
            if id in spotDict[ind]:
                count += 1
        percentage = (count/len(ids))
        cutEuDSimTf.append(percentage)

print(cutEuDSimTf)

20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
[0.05, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.25, 0.0, 0.05, 0.15, 0.1, 0.05, 0.2, 0.0, 0.2, 0.0, 0.05, 0.05, 0.2, 0.0, 0.15, 0.05, 0.0, 0.0, 0.2, 0.25, 0.0, 0.2, 0.15, 0.2, 0.05, 0.0, 0.0, 0.1, 0.05, 0.05, 0.25, 0.1, 0.1, 0.0, 0.0, 0.15, 0.2, 0.0, 0.05, 0.0,

In [164]:
matrix=CVectorizer()
im,reverse = CosSim(matrix)

noCutCosSimCV= []

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
    percentage = (count/20)
    noCutCosSimCV.append(percentage)

print(noCutCosSimCV)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [187]:
matrix=CVectorizer()
im,reverse = CosSim(matrix)

cutCosSimCv= []

allowedNames = list(indices.index)

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    names = wanted_dict['top10Names']
    if names[0] != 'wrong format':
        for name in names:
            if name not in allowedNames:
                i = names.index(name)
                ids.pop(i)
        count = 0
        print(len(ids))
        for id in ids:
            if id in spotDict[ind]:
                count += 1
        percentage = (count/len(ids))
        cutCosSimCv.append(percentage)

print(cutCosSimCv)

20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [165]:
matrix=TfidVectorizer()
im,reverse = CosSim(matrix)

noCutCosSimTf= []

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    count = 0
    for id in ids:
        if id in spotDict[ind]:
            count += 1
    percentage = (count/20)
    noCutCosSimTf.append(percentage)

print(noCutCosSimTf)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [186]:
matrix=TfidVectorizer()
im,reverse = CosSim(matrix)

cutCosSimTf= []

allowedNames = list(indices.index)

for ind in spotDict:
    name = indices.index[int(ind)]
    wanted_dict = get_recommendations(name,sim,reverse)
    ids = wanted_dict['top10']
    names = wanted_dict['top10Names']
    if names[0] != 'wrong format':
        for name in names:
            if name not in allowedNames:
                i = names.index(name)
                ids.pop(i)
        count = 0
        print(len(ids))
        for id in ids:
            if id in spotDict[ind]:
                count += 1
        percentage = (count/len(ids))
        cutCosSimTf.append(percentage)

print(cutCosSimTf)

1416
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [79]:
def Random_tester():
    rand= np.random.randint(0,len(df3),299)
    return rand

sample=Random_tester()
df_sample=pd.DataFrame(sample)
df_sample = df_sample.drop_duplicates()
# df_sample.to_csv("random.csv",index=False)

In [84]:
import requests
from time import sleep

apiKey = 'BQBO4yCnDwkL_i0m3h3TyQDkLE3j0le2VxgJdaMvhKyL7A6LZpsiNIMbsKIVSICsHBMD4PaCKQqTLDesOcSs5awkCm_zQA_6QkJP5TKRcl94CpyG-15eookOh89b6MXB1bASAcqe5oE8Y8fSr14-7R2ounycsws7t4lVwsi-QUW7QZATLfLorKQxV9NX2Kk'
BASE_URL = 'https://api.spotify.com/v1/'
headers = {
    'Authorization': 'Bearer {token}'.format(token=apiKey)
}


rand_df=pd.read_csv("random.csv",index_col=None)
# print(rand_df.head())
spotifyResultsDf1 = pd.DataFrame()
spotifyResultsDf2 = pd.DataFrame()
spotifyResultsDf3 = pd.DataFrame()
i=0
for item,row in rand_df.iterrows():
    id=str(df3['spotifyid'].iloc[row["Num"]])
    topTwentySpotArray = []
    r = requests.get(BASE_URL + 'artists/' + id + '/related-artists', headers=headers)
    rData = r.json()
    for entry in rData['artists']:
        spotId =  entry['id']
        topTwentySpotArray.append(spotId)
        
    if(len(topTwentySpotArray) < 20):
        while len(topTwentySpotArray) < 20:
            topTwentySpotArray.append("")

    print(i)
    i+=1
    numberWanted = str(row['Num'])
    if i < 100:
        spotifyResultsDf1[numberWanted] = topTwentySpotArray
    elif i >= 100 and i < 200:
        spotifyResultsDf2[numberWanted] = topTwentySpotArray
    else:
        spotifyResultsDf3[numberWanted] = topTwentySpotArray

    sleep(0.4)


spotifyResultsDf1.to_csv('spotify_results1.csv')
spotifyResultsDf2.to_csv('spotify_results2.csv')
spotifyResultsDf3.to_csv('spotify_results3.csv')



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
